In [2]:
# import the necessary libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [3]:
my_page = requests.get(
    "https://discovery.smartcolumbusos.com/?apiAccessible=true")
print(my_page.status_code) #200
#print(my_page.text) #prints whole output in byte format
#print(my_page.content) #prints whole output in unicode format

200


In [4]:
soup = BeautifulSoup(my_page.content, 'html.parser')

In [8]:
print (soup.prettify()[:])

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8">
   <meta content="width=device-width, initial-scale=1.0" name="viewport">
    <meta content="ie=edge" http-equiv="X-UA-Compatible">
     <title>
      Datasets
     </title>
     <link href="/favicon.ico" rel="shortcut icon">
      <link href="/2.a2f92c14aae5885c4b7f.css" rel="stylesheet">
       <link href="/0.a2f92c14aae5885c4b7f.css" rel="stylesheet"/>
      </link>
     </link>
    </meta>
   </meta>
  </meta>
 </head>
 <body>
  <section id="root">
  </section>
  <script src="/config.js" type="text/javascript">
  </script>
  <script src="/runtime.7680e7c1a7a03ac628f9.js" type="text/javascript">
  </script>
  <script src="/vendors.87491822db874fe28883.js" type="text/javascript">
  </script>
  <script src="/main.1da5c23e77dccdde435d.js" type="text/javascript">
  </script>
 </body>
</html>



In [117]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options


options = Options()
options.headless = True

EXE_PATH = r'/Applications/chromedriver'
driver = webdriver.Chrome(executable_path=EXE_PATH, chrome_options=options)
driver.set_window_size(1440, 900)
driver.get('https://discovery.smartcolumbusos.com/?apiAccessible=true')
time.sleep(5)
soup = driver.page_source

last_modified= []
file_type = []
description = []
url_final = []
title_final =[]


j=0
while j<22:
    url_temp = []
    title_temp = []


    for a in driver.find_elements_by_xpath('.//a'):
        url_temp.append(str(a.get_attribute('href')))
        title_temp.append(str(a.text))

    for a in driver.find_elements_by_xpath(".//div[@class='last-modified']"):
        last_modified.append(str(a.text))
    
    time.sleep(5)
    
    for a in driver.find_elements_by_xpath(".//span[@class='file-type']"):
        file_type.append(str(a.text))

    for a in driver.find_elements_by_xpath(".//div[@class='description']"):
        description.append(str(a.text))

    z=0
    for x, y in zip(url_temp[12:32], title_temp[12:32]):
        z+=1
        if z%2 == 0:
            url_final.append(x)
            title_final.append(y)
            
    y = driver.find_elements_by_tag_name("button")

    z=0
    for x in y:
        z+=1
        if z==9:
            x.click()
    time.sleep(5)
    j+=1
    if j%5==0:
        print(str(j)," pages scraped, ", str(len(url_final)), " datasets collected")



driver.close()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


5  pages scraped,  50  datasets collected
10  pages scraped,  100  datasets collected
15  pages scraped,  150  datasets collected
20  pages scraped,  200  datasets collected


In [118]:
print(len(url_final))
print(len(title_final))
print(len(file_type))
print(len(last_modified))
print(len(description))

217
217
214
214
214


In [122]:
url_final=url_final[:-3]
title_final=title_final[:-3]

In [123]:
df = pd.DataFrame({
    'Title':title_final,
    'File Type':file_type,
    'Last Modified Date':last_modified,
    'Description':description,
    'URL':url_final
}) 

In [145]:
agency = []
for x in df['URL']:
    y = x[46:]
    z=y[:y.find("/")]
    agency.append(z)
    
df['Agency']=agency

new_dates = []
for x in df['Last Modified Date']:
    y = x[:12]
    if y == "Date not pro":
        y=""
    new_dates.append(y)

df['Last Modified Date']=new_dates

In [146]:
df.head()

,Title,File Type,Last Modified Date,Description,URL,Agency
0,Age and Sex Demographics at Census Tract Level...,CSV,"May 2, 2018",This dataset identifies the age data categoriz...,https://discovery.smartcolumbusos.com/dataset/...,united_states_census_bureau
1,Aggregated Travel Time to Work - Central Ohio ...,CSV,"May 2, 2018",This dataset identifies the aggregated travel ...,https://discovery.smartcolumbusos.com/dataset/...,united_states_census_bureau
2,Behavioral Risk Factor Surveillance System (BR...,CSV,"Feb 26, 2018",2011 to present. BRFSS combined land line and ...,https://discovery.smartcolumbusos.com/dataset/...,center_for_disease_control
3,Bridge Inventory -- WGIS,CSV,"Apr 9, 2018",This dataset identifies the ODOT maintained br...,https://discovery.smartcolumbusos.com/dataset/...,odot
4,Cell Coverage Dark Spots - Columbus 2018 -- Ur...,CSV,"May 2, 2018",This Geotab dataset is intended to show where ...,https://discovery.smartcolumbusos.com/dataset/...,geotab


In [147]:
df.to_excel("Smart Columbus API data sources.xlsx", index=False)